In [6]:
pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.6 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 811.9 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.2/11.0 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.0 MB 12.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.7/11.0 MB 21.8 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/11.0 MB 31.9 MB/s eta 0:00:01
   ----------------------------- ---------- 8.0/11.0 MB 36.7 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 50.1 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 50.1 MB/s eta 0:00:01
   ----------------

In [3]:
pip install flask

  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ------------ --------------------------- 30.7/101.7 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 1.5 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached werkzeug-3.0.3-py3-none-any.whl (227 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install SQLAlchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 660.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.1 MB 1.3 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/2.1 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------  2.0/2.1 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/292.3 kB ? eta -:--:--
   ---------------------------------------- 292.3/292.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:

import pandas as pd
from flask import Flask, request, jsonify
import sqlite3
from sqlalchemy import create_engine
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import pickle


In [12]:
df = pd.read_csv(r"data.csv")
df

,Age,Education_Level,Occupation,Number_of_Dependents,Location,Work_Experience,Marital_Status,Employment_Status,Household_Size,Homeownership_Status,Type_of_Housing,Gender,Primary_Mode_of_Transportation,Income
0,56,Master's,Technology,5,Urban,21,Married,Full-time,7,Own,Apartment,Male,Public transit,72510
1,69,High School,Finance,0,Urban,4,Single,Full-time,7,Own,Apartment,Male,Biking,75462
2,46,Bachelor's,Technology,1,Urban,1,Single,Full-time,7,Own,Single-family home,Female,Car,71748
3,32,High School,Others,2,Urban,32,Married,Full-time,1,Own,Apartment,Female,Car,74520
4,60,Bachelor's,Finance,3,Urban,15,Married,Self-employed,4,Own,Townhouse,Male,Walking,640210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,58,High School,Others,0,Suburban,26,Divorced,Full-time,6,Own,Townhouse,Female,Car,74994
9996,23,Master's,Technology,3,Urban,16,Divorced,Full-time,1,Own,Single-family home,Female,Biking,72676
9997,41,Doctorate,Education,3,Urban,50,Single,Full-time,4,Own,Single-family home,Female,Public transit,2684820
9998,44,High School,Education,0,Urban,3,Married,Part-time,1,Own,Single-family home,Male,Biking,57955


In [13]:
df = df.drop(["Location",
            "Education_Level",
            "Occupation",
            "Marital_Status",
            "Employment_Status",
            "Homeownership_Status",
            "Type_of_Housing",
            "Gender",
            "Primary_Mode_of_Transportation"],
             axis=1)
df

,Age,Number_of_Dependents,Work_Experience,Household_Size,Income
0,56,5,21,7,72510
1,69,0,4,7,75462
2,46,1,1,7,71748
3,32,2,32,1,74520
4,60,3,15,4,640210
...,...,...,...,...,...
9995,58,0,26,6,74994
9996,23,3,16,1,72676
9997,41,3,50,4,2684820
9998,44,0,3,1,57955


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Age                   10000 non-null  int64
 1   Number_of_Dependents  10000 non-null  int64
 2   Work_Experience       10000 non-null  int64
 3   Household_Size        10000 non-null  int64
 4   Income                10000 non-null  int64
dtypes: int64(5)
memory usage: 390.8 KB


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop("Income", axis=1), df["Income"], test_size=0.2, random_state=42)

In [19]:
X_train_n = MinMaxScaler(X_train)
X_test_n = MinMaxScaler(X_test)

In [20]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [26]:
modelo = "modelo.sav"
pickle.dump(rfr, open(modelo, "wb"))
rfr = pickle.load("modelo.sav ")

In [29]:
import pandas as pd
from flask import Flask, request
import sqlite3
from sqlalchemy import create_engine
import numpy as np

churro = """sqlite:///dbname.db"""
engine = create_engine(churro)


df.to_sql("datos",con=engine, if_exists="replace", index=None)

10000

In [32]:
app = Flask(__name__)
app.config["DEBUG"] = True


In [33]:
@app.route('/predict', methods=['POST'])
def predict():
     json_ = request.json
     query_df = pd.DataFrame(json_)
     query = pd.get_dummies(query_df)
     prediction = rfr.predict(query)
     return jsonify({'prediction': list(prediction)})

In [34]:
# @app.route('/predict', methods=['POST']) # Your API endpoint URL would consist /predict
# def predict():
#     if rfr:
#         try:
#             json_ = request.json
#             query = pd.get_dummies(pd.DataFrame(json_))
#             query = query.reindex(columns=model_columns, fill_value=0)

#             prediction = list(rfr.predict(query))

#             return jsonify({'prediction': prediction})

#         except:

#             return jsonify({'trace': traceback.format_exc()})
#     else:
#         print ('Train the model first')
#         return ('No model here to use')

AssertionError: View function mapping is overwriting an existing endpoint function: predict